In [19]:
import yfinance as yf
import pandas as pd

ticker = "F"
ul_price = yf.Ticker(ticker).history(period='1d')['Close'][0]
expirations = yf.Ticker(ticker).options[:3]

calls = pd.concat( [yf.Ticker(ticker).option_chain(expirations[0]).calls, yf.Ticker(ticker).option_chain(expirations[1]).calls, yf.Ticker(ticker).option_chain(expirations[2]).calls])

calls['intrinsic_value'] = ul_price - calls['strike'] 
calls.loc[calls['intrinsic_value'] < 0, 'intrinsic_value'] = 0
calls['market_price'] = (calls['bid'] + calls['ask']) / 2
calls['extrinsic_value'] = calls['market_price'] - calls['intrinsic_value']
calls['extrinsic_to_strike_percentage'] = calls['extrinsic_value'] / calls['strike'] 
calls['extrinsic_to_underlying_percentage'] = calls['extrinsic_value'] / ul_price
calls['called_away_percentage'] = ( ul_price - calls['market_price'] - calls['strike']) / ul_price * -1

print(ul_price)
print(calls[calls['extrinsic_value'] > .1].to_markdown())

13.180000305175781
|    | contractSymbol   | lastTradeDate             |   strike |   lastPrice |   bid |   ask |   change |   percentChange |   volume |   openInterest |   impliedVolatility | inTheMoney   | contractSize   | currency   |   intrinsic_value |   market_price |   extrinsic_value |   extrinsic_to_strike_percentage |   extrinsic_to_underlying_percentage |   called_away_percentage |
|---:|:-----------------|:--------------------------|---------:|------------:|------:|------:|---------:|----------------:|---------:|---------------:|--------------------:|:-------------|:---------------|:-----------|------------------:|---------------:|------------------:|---------------------------------:|-------------------------------------:|-------------------------:|
|  8 | F230804C00013000 | 2023-07-31 15:07:30+00:00 |     13   |        0.31 |  0.3  |  0.31 |    -0.08 |       -20.5128  |     8466 |           7317 |            0.33985  | True         | REGULAR        | USD        |         

In [2]:
xxx = yf.Ticker(ticker).history(period='1d')['Close'][0]
print(xxx)

13.260000228881836


In [ ]:
import json
print(json.dumps(yf.Ticker(ticker).info, indent=4) )

In [15]:
import warnings
import datetime as dt
from pyblackscholesanalytics.market.market import MarketEnvironment
from pyblackscholesanalytics.options.options import PlainVanillaOption, DigitalOption

warnings.filterwarnings("ignore")

def option_factory(mkt_env, plain_or_digital, option_type, **kwargs):
    option_dispatcher = {
        "plain_vanilla": {"call": PlainVanillaOption(mkt_env, **kwargs),
                          "put": PlainVanillaOption(mkt_env, option_type="put", **kwargs)
                          },
        "digital": {"call": DigitalOption(mkt_env, **kwargs),
                    "put": DigitalOption(mkt_env, option_type="put", **kwargs)
                    }
    }

    return option_dispatcher[plain_or_digital][option_type]

def main():
    # if np_output is True, the output will be np.ndarray, otherwise pd.DataFrame
    np_output = False  # True

    # default market environment
    market_env = MarketEnvironment(dt.datetime.today(), 0.05, 3.39, 1.12)
    print(market_env)

     # define option style and type
    opt_style = "plain_vanilla"  # "digital" # "plain_vanilla"
    opt_type = "put"  # "put"
    option = option_factory(market_env, opt_style, opt_type, T='12-08-2023', K=3.5  )
    print(option)
    print(option.price())
    print(option.price_upper_limit())
    print(option.price_lower_limit())

main()

# Define the benchmark yield
bm_tickerSymbol = '^IRX'

# define the parameters as variables
S = 100 # underlying price
K = 110 # strike price
T = 0.5 # time to expiration (in years)
r = 0.05 # risk-free interest rate
q = 0 # dividend yield
sigma = 0.2 # volatility



Initializing the MarketEnvironment!
MarketEnvironment(t=29-07-2023, r=5.0%, S_t=3.4, sigma=112.0%)
Initializing the EuropeanOption!
Initializing the EuropeanOption!
Initializing the EuropeanOption!
Initializing the EuropeanOption!
PlainVanillaOption('put', S_t=3.4, K=3.5, t=29-07-2023, T=12-08-2023, tau=0.04y, r=5.0%, sigma=112.0%)
[0.34451337]
[3.49377267]
[0.10377267]


In [17]:
import yfinance as yf
import pandas as pd
import datetime # import the datetime module

ticker = yf.Ticker("HUT") 
commish = 1
today = pd.to_datetime(datetime.date.today()) # get today's date as a DatetimeArray object


exp_dates = ticker.options # get all expiration dates
max_puts = [] # create an empty list to store the max_put for each date
today_data = ticker.history(period='1d') # get today's data
close_price = today_data['Close'].iloc[0] # get the close price
for date in exp_dates:
  opt = ticker.option_chain(date) # get option chain for a given date
  
  puts = opt.puts[(opt.puts['bid'] > 0) & (opt.puts['openInterest'] > 0)].copy() # get put options data and exclude puts that have a bid of 0
  
  puts.loc[:, 'lastTradeDate'] = pd.to_datetime(puts['lastTradeDate']) # convert the lastTradeDate column to a datetime object using .loc method
  puts.loc[:, 'lastTradeDate'] = puts['lastTradeDate'].dt.tz_localize(None) # remove the timezone information using .dt.tz_localize(None) method
  puts = puts[puts['lastTradeDate'] >= today - pd.Timedelta(days=30)] # filter out the puts that have not been traded in the last 30 days using .loc method


  # calculate extrinsic value and find the highest one
  puts.loc[:, 'mid'] = (puts['bid'] + puts['ask']) / 2 # calculate the mid point between the bid and ask using .loc method
  puts.loc[:, 'intrinsic'] = puts['strike'] - close_price # calculate intrinsic value using the close price using .loc method
  puts.loc[:, 'intrinsic'] = puts['intrinsic'].clip(lower=0) # ensure that intrinsic value is not less than 0 using .clip method
  puts.loc[:, 'extrinsic'] = puts['mid'] - puts['intrinsic'] # calculate extrinsic value using the mid point using .loc method
  max_extrinsic = puts['extrinsic'].max() # find the maximum extrinsic value
  max_put = puts[puts['extrinsic'] == max_extrinsic] # find the put option with the maximum extrinsic value
  max_puts.append(max_put) # add the max_put to the list


max_puts_df = pd.concat(max_puts) # concatenate the list of dataframes into a single dataframe

ticker_len = len(ticker.ticker) # get the length of the ticker symbol

max_puts_df.loc[:, 'expiryDate'] = pd.to_datetime(max_puts_df['contractSymbol'].str.slice(ticker_len, ticker_len+6), format='%y%m%d').dt.date # create a column named expiryDate using the contractSymbol column and convert it to a date object
max_puts_df.loc[:, 'dte'] = (pd.to_datetime(max_puts_df['expiryDate']) - today).dt.days # create a column named dte using the expiryDate column and calculate the days to expiration as an int

max_puts_df = max_puts_df[~(max_puts_df['dte'] == 0)] # remove rows where dte is 0

max_puts_df.loc[:, 'simpleReturn'] = (max_puts_df['extrinsic'] - commish/100) / max_puts_df['strike'] # calculate the simple return using the extrinsicValue and strike columns
max_puts_df.loc[:, 'annualized'] = max_puts_df['simpleReturn'] / max_puts_df['dte'] * 365  # calculate the annualized return using the extrinsicValue and strike columns

xxx_df = max_puts_df.loc[:, ['dte', 'expiryDate', 'strike', 'volume', 'openInterest', 'lastTradeDate', 'lastPrice', 'bid', 'ask', 'mid', 'extrinsic', 'simpleReturn', 'annualized', 'impliedVolatility']]
print(xxx_df.to_markdown())

#print(max_puts_df.to_markdown()) # print the updated dataframe as a markdown table


|    |   dte | expiryDate   |   strike |   volume |   openInterest | lastTradeDate       |   lastPrice |   bid |   ask |   mid |   extrinsic |   simpleReturn |   annualized |   impliedVolatility |
|---:|------:|:-------------|---------:|---------:|---------------:|:--------------------|------------:|------:|------:|------:|------------:|---------------:|-------------:|--------------------:|
|  3 |     6 | 2023-08-04   |      3.5 |      820 |           1548 | 2023-07-28 19:59:16 |        0.2  |  0.2  |  0.21 | 0.205 |       0.185 |      0.05      |     3.04167  |            1.09375  |
|  3 |    13 | 2023-08-11   |      3.5 |       25 |            286 | 2023-07-28 19:50:52 |        0.33 |  0.3  |  0.35 | 0.325 |       0.305 |      0.0842857 |     2.36648  |            1.20313  |
|  5 |    20 | 2023-08-18   |      3.5 |       59 |           1035 | 2023-07-28 19:52:08 |        0.4  |  0.38 |  0.41 | 0.395 |       0.375 |      0.104286  |     1.90321  |            1.1875   |
|  4 |    27 | 